<a href="https://colab.research.google.com/github/ImadT/Classification-des-pistes-audio/blob/master/projet_finale__Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Drive      
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#imports
import os
import glob
import pandas as pd
import numpy as np
import librosa
import random
import time 
import pickle
import queue
import threading
import tensorflow as tf

import scipy.io.wavfile as wav
from scipy.fftpack import fft
from scipy import signal
from scipy.spatial.distance import squareform

from librosa.display import specshow, waveplot
import sklearn as skl
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV

import IPython.display as ipd

import matplotlib
import matplotlib.pyplot as plt

np.random.seed(42)

%matplotlib inline

In [0]:
from scipy import signal
from scipy.spatial.distance import cdist

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

import IPython.display as ipd

import matplotlib
import matplotlib.pyplot as plt

np.random.seed(42)

%matplotlib inline

pd.set_option('max_info_columns', 999)

def swish(x):
    return x*K.sigmoid(x)

get_custom_objects().update({'custom_activation': swish})

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout, Input, BatchNormalization, CuDNNLSTM
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import Callback, EarlyStopping,TensorBoard, ModelCheckpoint

from tensorflow.keras.metrics import top_k_categorical_accuracy

In [0]:
#import data for training
features = pd.read_csv("/content/drive/My Drive/projet IA/features.csv", header=[0, 1, 2], index_col=0)
classes = pd.read_csv("/content/drive/My Drive/projet IA/classes.csv", index_col=0)

In [0]:
#afficher matrice de correlation
df = pd.DataFrame()

result = pd.merge(features, classes['genres'], on='track_id')
result.corr()

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (3 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(chroma_cens, kurtosis, 01)","(chroma_cens, kurtosis, 02)","(chroma_cens, kurtosis, 03)","(chroma_cens, kurtosis, 04)","(chroma_cens, kurtosis, 05)","(chroma_cens, kurtosis, 06)","(chroma_cens, kurtosis, 07)","(chroma_cens, kurtosis, 08)","(chroma_cens, kurtosis, 09)","(chroma_cens, kurtosis, 10)","(chroma_cens, kurtosis, 11)","(chroma_cens, kurtosis, 12)","(chroma_cens, max, 01)","(chroma_cens, max, 02)","(chroma_cens, max, 03)","(chroma_cens, max, 04)","(chroma_cens, max, 05)","(chroma_cens, max, 06)","(chroma_cens, max, 07)","(chroma_cens, max, 08)","(chroma_cens, max, 09)","(chroma_cens, max, 10)","(chroma_cens, max, 11)","(chroma_cens, max, 12)","(chroma_cens, mean, 01)","(chroma_cens, mean, 02)","(chroma_cens, mean, 03)","(chroma_cens, mean, 04)","(chroma_cens, mean, 05)","(chroma_cens, mean, 06)","(chroma_cens, mean, 07)","(chroma_cens, mean, 08)","(chroma_cens, mean, 09)","(chroma_cens, mean, 10)","(chroma_cens, mean, 11)","(chroma_cens, mean, 12)","(chroma_cens, median, 01)","(chroma_cens, median, 02)","(chroma_cens, median, 03)","(chroma_cens, median, 04)",...,"(tonnetz, max, 05)","(tonnetz, max, 06)","(tonnetz, mean, 01)","(tonnetz, mean, 02)","(tonnetz, mean, 03)","(tonnetz, mean, 04)","(tonnetz, mean, 05)","(tonnetz, mean, 06)","(tonnetz, median, 01)","(tonnetz, median, 02)","(tonnetz, median, 03)","(tonnetz, median, 04)","(tonnetz, median, 05)","(tonnetz, median, 06)","(tonnetz, min, 01)","(tonnetz, min, 02)","(tonnetz, min, 03)","(tonnetz, min, 04)","(tonnetz, min, 05)","(tonnetz, min, 06)","(tonnetz, skew, 01)","(tonnetz, skew, 02)","(tonnetz, skew, 03)","(tonnetz, skew, 04)","(tonnetz, skew, 05)","(tonnetz, skew, 06)","(tonnetz, std, 01)","(tonnetz, std, 02)","(tonnetz, std, 03)","(tonnetz, std, 04)","(tonnetz, std, 05)","(tonnetz, std, 06)","(zcr, kurtosis, 01)","(zcr, max, 01)","(zcr, mean, 01)","(zcr, median, 01)","(zcr, min, 01)","(zcr, skew, 01)","(zcr, std, 01)",genres
"(chroma_cens, kurtosis, 01)",1.000000,0.333799,0.047314,0.183893,0.216923,0.209492,0.145475,0.223332,0.221560,0.172495,0.095915,0.377190,-0.127303,-0.130356,-0.042837,-0.061894,-0.119776,-0.095361,-0.087380,-0.096394,-0.111574,-0.078037,-0.059751,-0.161800,-0.245684,-0.043549,0.138818,0.130309,0.002077,0.012888,0.044245,0.019963,0.043331,0.086817,0.038617,-0.168105,-0.224816,-0.033636,0.147037,0.141442,...,-0.063563,-0.065267,-0.061504,-0.046875,-0.032625,-0.185505,-0.026727,-0.049212,-0.059399,-0.047406,-0.041224,-0.189094,-0.027284,-0.059348,0.020956,0.043772,0.114543,0.035711,0.076226,0.056630,0.005353,0.028870,0.035398,0.070806,0.030786,0.039132,-0.049924,-0.105854,-0.186637,-0.182373,-0.151990,-0.136600,-0.008573,-0.028872,0.033587,0.045052,-0.002953,-0.026875,-0.009319,-0.016925
"(chroma_cens, kurtosis, 02)",0.333799,1.000000,0.090431,0.294354,0.284165,0.172308,0.139419,0.178085,0.320730,0.167317,0.165494,0.312164,-0.162722,-0.094354,-0.163989,-0.064117,-0.089758,-0.117102,-0.110870,-0.166905,-0.092834,-0.118315,-0.077581,-0.081004,-0.033409,-0.083096,-0.024461,0.086235,0.093267,0.053371,0.004635,0.009804,0.007605,0.009321,0.035292,0.029344,-0.016502,-0.077879,-0.009411,0.083751,...,-0.070647,-0.051233,-0.022421,0.020044,0.060252,0.025944,0.003960,-0.006218,-0.019680,0.022852,0.062171,0.020743,0.005596,-0.012503,0.060582,0.079657,0.137756,0.139026,0.094728,0.070181,0.004967,0.027977,-0.049611,-0.038214,0.012357,0.033389,-0.098098,-0.099957,-0.187051,-0.196685,-0.160046,-0.144811,-0.009148,-0.030996,0.058468,0.068913,0.014279,-0.042154,-0.003265,-0.041775
"(chroma_cens, kurtosis, 03)",0.047314,0.090431,1.000000,0.136519,0.061075,0.036194,0.066842,0.076790,0.038855,0.053856,0.030409,0.040836,-0.004493,-0.056530,-0.080601,-0.052420,-0.008835,-0.013407,-0.041113,-0.076965,-0.020840,-0.008470,-0.011892,-0.004471,0.023740,-0.041048,-0.097257,-0.062046,0.010339,0.019579,-0.023322,-0.037417,-0.003811,0.031510,0.036141,0.036039,0.018671,-0.049200,-0.085218,-0.061595,...,-0.000173,0.007312,-0.015577,-0.011694,-0.038698,0.053015,0.028745,0.0

In [0]:
result.to_csv("/content/drive/My Drive/projet IA/result.csv")

# Models

In [0]:
#to fit the output
label = pd.read_csv("/content/drive/My Drive/projet IA/classes.csv", index_col = 0)

enc = LabelBinarizer()
y_enc = enc.fit_transform(label.genres.values.reshape(-1, 1)) # .title

In [0]:
y_enc

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [0]:
from sklearn.decomposition import PCA
#This estimator scales and translates each feature individually such that it is in the given range on the training set, i.e. between zero and one.
X = MinMaxScaler().fit_transform(features)
# phase ACP
pca = PCA(n_components=0.95)
new_X = pca.fit_transform(X)

In [0]:
new_X.shape

(8000, 148)

In [0]:
#split of data
X_train, X_test, y_train, y_test, = train_test_split(new_X, y_enc, test_size=0.20, random_state=42)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout, Input, BatchNormalization, CuDNNLSTM
from keras.models import Model, load_model
from keras.callbacks import Callback, EarlyStopping
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.metrics import top_k_categorical_accuracy

In [0]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax'))
#relu

model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               19072     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='Nadam', 
              metrics=["binary_crossentropy", "accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
checkpoint_callback = ModelCheckpoint('/content/drive/My Drive/projet IA/models_crnn_weights.best.h5', monitor='val_acc', verbose=1,
                                          save_best_only=True, mode='max')
    
reducelr_callback = ReduceLROnPlateau(
                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,
                verbose=1
            )

callbacks_list = [checkpoint_callback, reducelr_callback]
"""
history = model.fit(X_train, y_enc_train, 
                      epochs=25, 
                      batch_size=100, 
                      verbose=1, 
                      validation_data=(X_test, y_enc_test)) """

history = model.fit(X_train, y_train, batch_size=100, epochs=25,
                        validation_data=(X_test, y_test), verbose=1, callbacks=callbacks_list)


"""
checkpoint_callback = ModelCheckpoint('./models/crnn/weights.best.h5', monitor='val_acc', verbose=1,
                                          save_best_only=True, mode='max')
    
    reducelr_callback = ReduceLROnPlateau(
                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,
                verbose=1
            )
    callbacks_list = [checkpoint_callback, reducelr_callback]

    # Fit the model and get training history.
    print('Training...')
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT,
                        validation_data=(x_val, y_val), verbose=1, callbacks=callbacks_list)
"""




Train on 6400 samples, validate on 1600 samples
Epoch 1/25





6400/6400 [==============================] - 10s 2ms/step - loss: 0.3656 - binary_crossentropy: 0.3656 - acc: 0.8753 - val_loss: 0.3369 - val_binary_crossentropy: 0.3369 - val_acc: 0.8749

Epoch 00001: val_acc improved from -inf to 0.87492, saving model to /content/drive/My Drive/projet IA/models_crnn_weights.best.h5
Epoch 2/25
6400/6400 [==============================] - 0s 51us/step - loss: 0.3259 - binary_crossentropy: 0.3259 - acc: 0.8788 - val_loss: 0.2828 - val_binary_crossentropy: 0.2828 - val_acc: 0.8915

Epoch 00002: val_acc improved from 0.87492 to 0.89148, saving model to /content/drive/My Drive/projet IA/models_crnn_weights.best.h5
Epoch 3/25
6400/6400 [==============================] - 0s 53us/step - loss: 0.2959 - binary_crossentropy: 0.2959 - acc: 0.8877 - val_loss: 0.2653 - val_binary_crossentropy: 0.2653 - val_acc: 0.8991

Epoch 00003: val_acc improved from 0.89148 to 0.89914, saving model to /content/d

"\ncheckpoint_callback = ModelCheckpoint('./models/crnn/weights.best.h5', monitor='val_acc', verbose=1,\n                                          save_best_only=True, mode='max')\n    \n    reducelr_callback = ReduceLROnPlateau(\n                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,\n                verbose=1\n            )\n    callbacks_list = [checkpoint_callback, reducelr_callback]\n\n    # Fit the model and get training history.\n    print('Training...')\n    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT,\n                        validation_data=(x_val, y_val), verbose=1, callbacks=callbacks_list)\n"

In [0]:
history

In [0]:
score = model.evaluate(
	x=X_test,
	y=y_test)

1600/1600 [==============================] - 0s 44us/step


In [0]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.24675972729921342
Test accuracy: 0.24675972729921342


# Nouveau modele 2_ model performant

In [0]:
def top1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=1)

def top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top3(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top4(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=4)

class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.epoch_time_start = time.time()
        self.times = []
        self.losses = []
        self.val_losses = []
        self.top1 = []
        self.top2 = []
        self.top3 = []
        self.top4 = []
        self.val_top1 = []
        self.val_top2 = []
        self.val_top3 = []
        self.val_top4 = []
        
    def on_epoch_end(self, batch, logs={}):
        self.val_losses.append(logs.get('val_loss'))
        self.val_top1.append(logs.get('val_top1'))
        self.val_top2.append(logs.get('val_top2'))
        self.val_top3.append(logs.get('val_top3'))
        self.val_top4.append(logs.get('val_top4'))
    
    def on_batch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        self.losses.append(logs.get('loss'))
        self.top1.append(logs.get('top1'))
        self.top2.append(logs.get('top2'))
        self.top3.append(logs.get('top3'))
        self.top4.append(logs.get('top4'))
        
    def convert_to_dict(self):
        return {
            "time" : self.times,
            "loss" : self.losses,
            "val_loss" : self.val_losses,
            "top1" : self.top1,
            "val_top1" : self.val_top1,
            "top2" : self.top2,
            "val_top2" : self.val_top2,
            "top3" : self.top3,
            "val_top3" : self.val_top3,
            "top4" : self.top4,
            "val_top4" : self.val_top4,
        }

In [0]:
label = pd.read_csv("/content/drive/My Drive/projet IA/classes.csv", index_col = 0)

#A simple way to extend these algorithms to the multi-class classification case is to use the so-called one-vs-all scheme.
#Methods
#	fit() >> Fit label binarizer
# fit_transform() >> Fit label binarizer and transform multi-class labels to binary labels.
enc = LabelBinarizer()
_enc = enc.fit(label.title.values.reshape(-1, 1))
import pickle
pickle.dump(_enc, open(r'/content/drive/My Drive/projet IA/_saved_enc','wb'))
y_enc = _enc.transform(label.title.values.reshape(-1, 1))

In [0]:
X = label.index.values
scaler = MinMaxScaler()
_features = pd.read_csv("/content/drive/My Drive/projet IA/features.csv", header=[0, 1, 2], index_col=0)
#This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.
_scaler = scaler.fit(_features[_features.columns])

import pickle
pickle.dump(_scaler, open(r'/content/drive/My Drive/projet IA/_saved_scaler','wb'))

_features[_features.columns] = _scaler.transform(_features[_features.columns])
_features

feature    chroma_cens                      ...       zcr                    
statistics    kurtosis                      ...       min      skew       std
number              01        02        03  ...        01        01        01
track_id                                    ...                              
2             0.201184  0.086361  0.003998  ...  0.000000  0.133702  0.186725
5             0.052194  0.021311  0.002979  ...  0.000000  0.136344  0.134086
10            0.123288  0.018694  0.007750  ...  0.000000  0.170807  0.121202
140           0.052329  0.014617  0.001425  ...  0.011429  0.160630  0.174871
141           0.044252  0.018773  0.001264  ...  0.017143  0.187712  0.082691
...                ...       ...       ...  ...       ...       ...       ...
154308        0.025204  0.012084  0.002196  ...  0.011429  0.257308  0.259237
154309        0.027278  0.014480  0.003143  ...  0.051429  0.168225  0.402619
154413        0.035576  0.008409  0.004902  ...  0.005714  0.175771  0.067319
154414        0.029466  0.010942  0.002972  ...  0.020000  0.182423  0.136856
155066        0.041370  0.018581  0.003100  ...  0.000000  0.327822  0.089425

[8000 rows x 518 columns]

In [0]:
# phase ACP
pca = PCA(n_components=0.95)
# Fit the model with X and apply the dimensionality reduction on X
pc = pca.fit(_features)
import pickle
pickle.dump(pc, open(r'/content/drive/My Drive/projet IA/_saved_pca','wb'))
new_X = pc.transform(_features)

In [0]:
new_X.shape

(8000, 148)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y_enc, test_size=0.20, random_state=42)

In [0]:
#used whene we load the model 
# X_train.shape[1] = 148

def create_model():
  model = Sequential()
  model.add(Dense(128, input_dim=148, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(8, activation='softmax'))

  model.compile(loss='binary_crossentropy', 
              optimizer='Nadam', 
              metrics=["binary_crossentropy", top1, top2, top3, top4])

  return model

In [0]:
X_train.shape[1]

148

In [0]:
#the model 
"""
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax'))
"""
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               19072     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)               

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='Nadam', 
              metrics=["binary_crossentropy", top1, top2, top3, top4])

In [0]:
X_train.shape

(6400, 148)

In [0]:
checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/projet IA/fully_connected_MLP_VF_weights_test.hdf5", verbose=1, save_best_only=True)
#model.fit(X_train, y_train, nb_epoch=20, batch_size=16, show_accuracy=True, validation_split=0.2, verbose=2, callbacks=[checkpointer])

#def load_trained_model(weights_path):
#   model = create_model()
#   model.load_weights(weights_path)

# a revoire il est possible de diminuer l'epochs a 100
time_callback = TimeHistory()
model.fit(X_train, y_train, 
          epochs=150, 
          batch_size=100, 
          verbose=1, 
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, time_callback])

Train on 6400 samples, validate on 1600 samples
Epoch 1/150
6400/6400 [==============================] - 1s 129us/step - loss: 0.3735 - binary_crossentropy: 0.3735 - top1: 0.1578 - top2: 0.3000 - top3: 0.4383 - top4: 0.5731 - val_loss: 0.3576 - val_binary_crossentropy: 0.3576 - val_top1: 0.3481 - val_top2: 0.5225 - val_top3: 0.6344 - val_top4: 0.7538

Epoch 00001: val_loss improved from inf to 0.35761, saving model to /content/drive/My Drive/projet IA/fully_connected_MLP_VF_weights_test.hdf5
Epoch 2/150
6400/6400 [==============================] - 0s 60us/step - loss: 0.3530 - binary_crossentropy: 0.3530 - top1: 0.2573 - top2: 0.4441 - top3: 0.5856 - top4: 0.7155 - val_loss: 0.3254 - val_binary_crossentropy: 0.3254 - val_top1: 0.3619 - val_top2: 0.5819 - val_top3: 0.7194 - val_top4: 0.8094

Epoch 00002: val_loss improved from 0.35761 to 0.32544, saving model to /content/drive/My Drive/projet IA/fully_connected_MLP_VF_weights_test.hdf5
Epoch 3/150
6400/6400 [============================

In [0]:
model.evaluate(X_test,y_test)

1600/1600 [==============================] - 0s 65us/step


[0.341375994682312, 0.341375994682312, 0.55875, 0.683125, 0.79375, 0.859375]

In [0]:
"""
_json = model.to_json()
# Save the json on a file
import json
with open('/content/drive/My Drive/projet IA/personal.json', 'w') as json_file:
    json.dump(_json, json_file)
model.save_weights("/content/drive/My Drive/projet IA/weight.h5",save_format="h5")
"""

In [0]:
"""with open('personal.json') as json_file:
    data = json.load(json_file)
__model = keras.models.model_from_json(data)
__model.load_weights("/content/weight.h5")"""

In [0]:
# returns accuracy
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [0]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Accuracy is: 55.875


In [0]:
score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[2])

1600/1600 [==============================] - 0s 54us/step
Test loss: 0.341375994682312
Test accuracy: 0.55875


**Save & loaD** model

In [0]:
#model saving
#model.save('/content/drive/My Drive/projet IA/fully_connected_MLP_VF.h5')  # creates a HDF5 file 'my_model.h5'

In [0]:
#load le model 

from keras.models import load_model
model_load = create_model()
model_load.load_weights("/content/drive/My Drive/projet IA/fully_connected_MLP_VF_weights_test.hdf5")
"""
with open('personal.json') as json_file:
    data = json.load(json_file)
model_load = keras.models.model_from_json(data)
model_load.load_weights("/content/weight.h5")
"""

'\nwith open(\'personal.json\') as json_file:\n    data = json.load(json_file)\nmodel_load = keras.models.model_from_json(data)\nmodel_load.load_weights("/content/weight.h5")\n'

In [0]:
# evaluate the model
model_load.evaluate(X_test,  y_test, verbose=2)

[0.24769114583730698, 0.24769114583730698, 0.5825, 0.744375, 0.82375, 0.8825]

# USE THE MODEL TO PREDICT

In [0]:
# example to use the load model
y_pred = model_load.predict(X_train[200:201])
print(y_pred, "============", y_train[200])

[[7.8602323e-05 2.0966506e-02 2.1169140e-04 2.3223080e-05 1.2562417e-03
  5.3444499e-05 8.4784500e-02 8.9262581e-01]] ============ [0 0 0 0 0 0 0 1]


In [0]:
tag_labels = enc.classes_
"""
tag_labels >>>>> 
array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype='<U13')
"""
predicted_tags = tag_labels[np.argmax(y_pred)]
print (predicted_tags)

Rock


In [0]:
_predicted_tags = tag_labels[np.argmax(y_train[200])]
print (_predicted_tags)

Rock


# EXTRACTION FEATURES FROM AUDIO 

In [0]:
import os
import multiprocessing
import warnings

from tqdm import tqdm
import numpy as np
from scipy import stats
import pandas as pd
import librosa


# the fonction returns the columns how the features.csv is structured 
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


columns()    

MultiIndex([('chroma_cens', 'kurtosis', '01'),
            ('chroma_cens', 'kurtosis', '02'),
            ('chroma_cens', 'kurtosis', '03'),
            ('chroma_cens', 'kurtosis', '04'),
            ('chroma_cens', 'kurtosis', '05'),
            ('chroma_cens', 'kurtosis', '06'),
            ('chroma_cens', 'kurtosis', '07'),
            ('chroma_cens', 'kurtosis', '08'),
            ('chroma_cens', 'kurtosis', '09'),
            ('chroma_cens', 'kurtosis', '10'),
            ...
            (    'tonnetz',      'std', '04'),
            (    'tonnetz',      'std', '05'),
            (    'tonnetz',      'std', '06'),
            (        'zcr', 'kurtosis', '01'),
            (        'zcr',      'max', '01'),
            (        'zcr',     'mean', '01'),
            (        'zcr',   'median', '01'),
            (        'zcr',      'min', '01'),
            (        'zcr',     'skew', '01'),
            (        'zcr',      'std', '01')],
           names=['feature', 'statistics', 

In [0]:
def compute_features(filepath):

    features = pd.Series(index=columns(), dtype=np.float32, name=filepath)

    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        #filepath = utils.get_audio_path(os.environ.get('AUDIO_DIR'), tid)
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rmse(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}: {}'.format(filepath, repr(e)))

    return features

In [0]:
compute_features("/content/drive/My Drive/projet IA/000002.mp3")

feature      statistics  number
chroma_cens  kurtosis    01        5.619328
                         02       -0.464132
                         03        1.627383
                         04       -0.274711
                         05        0.190102
                                     ...   
zcr          mean        01        0.081016
             median      01        0.079102
             min         01        0.028320
             skew        01        0.908157
             std         01        0.020830
Name: /content/drive/My Drive/projet IA/000010.mp3, Length: 518, dtype: float32

In [0]:
array_2 = compute_features("/content/drive/My Drive/projet IA/000005.mp3")

In [0]:
array_2

feature      statistics  number
chroma_cens  kurtosis    01       -0.171996
                         02        0.226217
                         03       -0.099524
                         04        0.325517
                         05        1.742335
                                     ...   
zcr          mean        01        0.059330
             median      01        0.047852
             min         01        0.002930
             skew        01        2.091125
             std         01        0.047794
Name: /content/drive/My Drive/projet IA/000005.mp3, Length: 518, dtype: float32

In [0]:
from sklearn.decomposition import PCA
#transformer la serie array_2 en dataframe
df_2 = pd.DataFrame(array_2).transpose()
#print("transpose\n",df_2)

# pour enfin appliquer la transformation a df_2
"""
_scaler = MinMaxScaler()
_features = pd.read_csv("/content/drive/My Drive/projet IA/features.csv", header=[0, 1, 2], index_col=0)
_features[_features.columns] = _scaler.fit_transform(_features[_features.columns])
"""
loaded_scaler = pickle.load(open('/content/drive/My Drive/projet IA/_saved_scaler','rb'))

#obj = _scaler.fit(_features)

df_2[df_2.columns] = loaded_scaler.transform(df_2[df_2.columns])
df_2


feature                                      chroma_cens  ...       zcr
statistics                                      kurtosis  ...       std
number                                                01  ...        01
/content/drive/My Drive/projet IA/000005.mp3    0.036528  ...  0.143396

[1 rows x 518 columns]

In [0]:
"""
_pca = PCA(n_components=0.95)
new_X = _pca.fit_transform(_features)
# Apply dimensionality reduction to X.
# X is projected on the first principal components previously extracted from a training set.
df_2_pca = _pca.transform(df_2)
"""
_pca = pickle.load(open('/content/drive/My Drive/projet IA/_saved_pca','rb'))
df_2_pca = _pca.transform(df_2)

In [0]:
#pd.DataFrame(array_2).transpose()
df_2_pca.shape

(1, 148)

In [0]:
y_pred = model_load.predict(df_2_pca)

loaded_enc = pickle.load(open('/content/drive/My Drive/projet IA/_saved_enc','rb'))
#df_2_pca = _pca.transform(df_2)
tag_labels = loaded_enc.classes_
"""
tag_labels >>>>> return 
array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype='<U13')
"""

# np.argmax(_list) Returns the indices of the maximum values along an axis.
predicted_tags = tag_labels[np.argmax(y_pred)]
print (predicted_tags)

Hip-Hop


In [0]:
tag_labels

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype='<U13')

In [0]:
classes

,genres,title
track_id,,
2,21,Hip-Hop
5,21,Hip-Hop
10,10,Pop
140,17,Folk
141,17,Folk
...,...,...
154308,21,Hip-Hop
154309,21,Hip-Hop
154413,10,Pop
